In [ ]:
import pixiedust

In [ ]:
pixiedust.enableJobMonitor()

In [ ]:
# Print Spark info and create sql_context
print('Spark Version: {0}'.format(sc.version))
print('Python Version: {0}'.format(sc.pythonVer))
print('Application Name: {0}'.format(sc.appName))
print('Application ID: {0}'.format(sc.applicationId))
print('Spark Master: {0}'.format( sc.master))

sql_context = SQLContext(sc, sparkSession=spark)

In [ ]:
%%bash
s3-dist-cp -D mapreduce.task.timeout=60000000 --src=s3://geowave-gpx-data/gpx --dest=hdfs://$HOSTNAME:8020/tmp/ 

In [ ]:
%%bash
/opt/accumulo/bin/accumulo shell -u root -p secret -e "importtable geowave.germany_gpx_SPATIAL_IDX /tmp/spatial"
/opt/accumulo/bin/accumulo shell -u root -p secret -e "importtable geowave.germany_gpx_GEOWAVE_METADATA /tmp/metadata"

In [ ]:
%%bash
# clear out potential old runs
#geowave config rmstore germany_osm
#geowave config rmstore germany_gpx_accumulo

# configure geowave connection params for name stores "germany_gpx_accumulo" and "kmeans_hbase"
#geowave config addstore germany_gpx_accumulo --gwNamespace geowave.germany_gpx -t accumulo --zookeeper $HOSTNAME:2181 --instance accumulo --user root --password secret
#geowave config addstore germany_osm --gwNamespace geowave.germany_osm -t accumulo --zookeeper $HOSTNAME:2181 --instance accumulo --user root --password secret

In [ ]:
#grab classes from jvm
accumulo_options_class = sc._jvm.mil.nga.giat.geowave.datastore.accumulo.operations.config.AccumuloRequiredOptions
query_options_class = sc._jvm.mil.nga.giat.geowave.core.store.query.QueryOptions
geowave_rdd_class = sc._jvm.mil.nga.giat.geowave.analytic.spark.GeoWaveRDD
sf_df_class = sc._jvm.mil.nga.giat.geowave.analytic.spark.sparksql.SimpleFeatureDataFrame
byte_array_class = sc._jvm.mil.nga.giat.geowave.core.index.ByteArrayId

#grab classes for spatial join
stage_command_class = sc._jvm.mil.nga.giat.geowave.cli.osm.operations.StageOSMToHDFSCommand
ingest_command_class = sc._jvm.mil.nga.giat.geowave.cli.osm.operations.IngestOSMToHDFSCommand
spatial_join_class = sc._jvm.mil.nga.giat.geowave.analytic.spark.spatial.SpatialJoin
config_hdfs_command_class = sc._jvm.mil.nga.giat.geowave.mapreduce.operations.ConfigHDFSCommand
config_options_class = sc._jvm.mil.nga.giat.geowave.core.cli.operations.config.options.ConfigOptions

In [ ]:
#setup input datastore
input_store = accumulo_options_class()
input_store.setInstance('accumulo')
input_store.setUser('root')
input_store.setPassword('secret')
input_store.setZookeeper(os.environ['HOSTNAME'] + ':2181')
input_store.setGeowaveNamespace('geowave.germany_gpx')

#Setup osm datastore
osm_store = accumulo_options_class()
osm_store.setInstance('accumulo')
osm_store.setUser('root')
osm_store.setPassword('secret')
osm_store.setZookeeper(os.environ['HOSTNAME'] + ':2181')
osm_store.setGeowaveNamespace('geowave.germany_osm')

input_store_plugin = input_store.createPluginOptions()
osm_store_plugin = osm_store.createPluginOptions()

In [ ]:
input_file = "file://~/germany-latest.osm.pbf"
hdfs_path = "hdfs:///"

osm_stage_path = hdfs_path + "osm_stage/"

config_file = ConfigOptions.getDefaultPropertyFile()
operation_params = manual_operations_class()
operation_params.getContext().put(ConfigOptions.PROPERTIES_FILE_CONTEXT, config_file)

config_command = config_hdfs_command_class()
config_command.setHdfsUrlParameter(hdfs_path)
config_command.execute(operation_params)

In [ ]:
#Ingest OSM data for germany to accumulo
stage = stage_command_class();
stage.setParameters(input_file, osm_stage_path);
stage.execute(operation_params);

In [ ]:
ingest = ingest_command_class();
ingest.setParameters(hdfs_path, null);

ingest.setInputStoreOptions(osm_store_plugin);

ingest.getIngestOptions().setJobName("GermanyIngest");

#ingest nodes
ingest.getIngestOptions().setMapperType("NODE");
ingest.execute(operation_params);

#ingest ways
ingest.getIngestOptions().setMapperType("WAY");
ingest.execute(operation_params);

#ingest relations
ingest.getIngestOptions().setMapperType("RELATION");
ingest.execute(operation_params);

In [ ]:
spatial_join = spatial_join_class()

spatial_join.performJoin(sc.sc(),input_store_plugin, null, osm_store_plugin, null)